Wstęp do sztucznej inteligencji. Lista 3. Zadanie 1 i 2
Karol Janic. Maj 2023. **Sieć neuronowa rozpoznająca ręcznie pisane cyfry**. 

In [ ]:
import tensorflow as tf
from tensorflow import keras

from google.colab import drive
import os

import cv2 as cv

Połączenie się z dyskiem Google i przejście do folderu docelowego

In [ ]:
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/wsi2023')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import zbioru danych. Podział na próbki uczące i testowe

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

Normalizacja obrazów. Przeskalowanie liczb opisujących kolor z zakresu [0,255] na [0,1]

In [ ]:
train_images = train_images.reshape(-1, 28 * 28) / 255.0
test_images = test_images.reshape(-1, 28 * 28) / 255.0

Stworzenie modelu sekwencyjnej sieci neuronowej o 3 warstwach:


1.   Warstwa gęsta o 1024 neuronach z funkją aktywcji ReLU, przyjmuje wektor o długości 28 * 28 = 784 opisujący piksele obrazu
2.   Warstwa Dropout, która wyłącza 40% losowych neuronów(w trakcie treningu) w celu zapobieżenia przetrenowaniu sieci
3.   Warstwa generująca wynik sieci

In [ ]:
def create_model():
  model = tf.keras.Sequential([
    keras.layers.Dense(1024, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

  return model

Trenowanie sieci przez 10 epok i zapis jej wyników do pliku

In [ ]:
model = create_model()

hist = model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels))  

model.save_weights('wsi3_zad1_weights')

Epoch 1/10
1875/1875 [==============================] - 25s 13ms/step - loss: 0.2241 - sparse_categorical_accuracy: 0.9334 - val_loss: 0.1042 - val_sparse_categorical_accuracy: 0.9675
Epoch 2/10
1875/1875 [==============================] - 24s 13ms/step - loss: 0.1115 - sparse_categorical_accuracy: 0.9650 - val_loss: 0.0785 - val_sparse_categorical_accuracy: 0.9758
Epoch 3/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.0853 - sparse_categorical_accuracy: 0.9731 - val_loss: 0.0717 - val_sparse_categorical_accuracy: 0.9769
Epoch 4/10
1875/1875 [==============================] - 25s 13ms/step - loss: 0.0714 - sparse_categorical_accuracy: 0.9772 - val_loss: 0.0669 - val_sparse_categorical_accuracy: 0.9787
Epoch 5/10
1875/1875 [==============================] - 25s 13ms/step - loss: 0.0604 - sparse_categorical_accuracy: 0.9806 - val_loss: 0.0633 - val_sparse_categorical_accuracy: 0.9814
Epoch 6/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.

Ładowanie zapisanego modelu

In [ ]:
model = create_model()
model.load_weights('wsi3_zad1_weights')

Ocena modelu na danych testowych

In [ ]:
loss, accuracy = model.evaluate(test_images, test_labels)

print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

313/313 [==============================] - 1s 3ms/step - loss: 0.0739 - sparse_categorical_accuracy: 0.9813
Test Loss: 0.07394345104694366
Test Accuracy: 0.9812999963760376


Ocena modelu na podstawie własnoręcznie przygotowanych próbek

In [ ]:
DIGITS = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
SAMPLES_NUMBER = 10

successes = 0
failures = {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []}

for digit in DIGITS:
  for k in range(SAMPLES_NUMBER):
    img = cv.imread(f'/content/drive/MyDrive/Colab Notebooks/wsi2023/test_digits/{digit}.{k}.png', cv.IMREAD_GRAYSCALE)
    img = img.astype('float32')
    img = img.reshape(1, 28*28, 1)
    img = 255 - img
    img /= 255

    pred = model.predict(img)
    if pred.argmax(axis=1)[0] == digit:
      successes += 1
    else:
      failures[digit].append(pred.argmax(axis=1)[0])
      #failures.append(f'/content/drive/MyDrive/Colab Notebooks/wsi2023/test_digits/{digit}.{k}.png')

print("Accuracy:", successes / len(DIGITS) * SAMPLES_NUMBER)

1/1 [==============================] - 0s 67ms/step
Accuracy: 60.0


In [ ]:
for digit, predicted in failures.items():
  print(f'{digit}: {(10 - len(predicted)) / 10 * 100} %   predicted = {predicted}')

0: 50.0 %   predicted = [2, 6, 3, 6, 5]
1: 30.0 %   predicted = [7, 7, 8, 7, 2, 4, 6]
2: 100.0 %   predicted = []
3: 90.0 %   predicted = [2]
4: 80.0 %   predicted = [7, 3]
5: 80.0 %   predicted = [6, 3]
6: 70.0 %   predicted = [5, 5, 5]
7: 30.0 %   predicted = [3, 2, 2, 2, 2, 2, 2]
8: 60.0 %   predicted = [3, 5, 6, 5]
9: 10.0 %   predicted = [5, 8, 0, 8, 5, 5, 3, 8, 8]


Dokładność na próbkach treningowych: 98%
Dokładność na własnych obrazach: 40%

Spadek dokładności wynika z innego charakteru pisma, który był w danych testowych i na własnych obrazach.
Sieć najgorzej rozpoznaje cyfrę 1, 7, 9. Najlepiej radzi sobie z 2, 4, 5.